In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")

import pandas as pd
from numpy.random import random
import datetime as dt
from statsmodels.tsa.stattools import acf, pacf
import statsmodels.api as sm
from scipy.signal import periodogram
import math

from pytsviz.viz import *

In [ ]:
dataset_size=366

In [ ]:
n_features = 3

Target series

In [ ]:
t_axis = pd.date_range(start='1/1/2020', periods = dataset_size, freq="D", name="t")

In [ ]:
y = (1 + random(dataset_size)) * np.linspace(6, 9, dataset_size) + \
    (1 + random(dataset_size)) * np.sin(np.linspace(0, 10*np.pi, dataset_size)) + \
    (1.5 + random(dataset_size)) * np.cos(np.linspace(0, 5.4*np.pi, dataset_size))
ts = pd.Series(index=t_axis, data=y, name="y")
ts_df = ts.to_frame()

Features

In [ ]:
feat_matrix = random((n_features, dataset_size))
feat_df = pd.DataFrame(index=t_axis, data={f"x_{i}": feat_matrix[i] for i in range(n_features)})

Forecast

In [ ]:
ext_t_axis = pd.date_range(start='2/1/2020', periods = dataset_size, freq="D", name="t")

In [ ]:
y_hat = y * (1 + 0.1 * random())

forecast = pd.Series(index=ext_t_axis, data=y_hat, name="y_hat")
forecast_df = forecast.to_frame()

In [ ]:
c_lower = [y_hat[i] - (0.001 * i) ** 1.2 for i in range(len(y_hat))]
c_upper = [y_hat[i] + (0.001 * i) ** 1.2 for i in range(len(y_hat))]
confidence_df = pd.DataFrame(index=ext_t_axis, data={"lower confidence": c_lower, "upper confidence": c_upper})

In [ ]:
total_df = pd.concat([ts, feat_df, forecast_df, confidence_df], axis = 1)

In [ ]:
total_df

In [ ]:
fig = plot_ts(total_df, y_cols=["y", "x_0"], tf="log", tf_args = (np.e,), keep_original=True)

In [ ]:
func1 = lambda x : x.isocalendar().week
func2 = lambda x : x.isocalendar().day

In [ ]:
plot_seasonal_ts(ts_df, period="month", subplots=True)

In [ ]:
plot_decomposed_ts(ts_df, method = "STL", subplots = True)

In [ ]:
inverted_df = pd.DataFrame(total_df.values[::-1], total_df.index, total_df.columns)
#forecast_plot(inverted_df, ts_col = "y", fc_cols = ["y_hat"], lower_col = "lower confidence", upper_col = "upper confidence")

y_hat_2 = pd.Series(data=y_hat*0.9, index = ext_t_axis, name = "y_hat_2")
multiple_fc_df = pd.concat([total_df, y_hat_2], axis = 1)
plot_forecast(multiple_fc_df, y_col = "y", fc_cols = ["y_hat"], upper_col = "upper confidence", lower_col = "lower confidence")

In [ ]:
plot_scatter_matrix(total_df, var1 = "x_0", var2="y", lags1 = [5, 10, 15])

In [ ]:
plot_scatter_fit(total_df, "x_0", "x_1", fit = "summary")

In [ ]:
plot_acf(ts_df, show_threshold = True)

In [ ]:
plot_acf(ts_df, partial=True, alpha = 0.1, show_threshold = True)

In [ ]:
plot_psd(ts_df, scaling="spectrum")

In [ ]:
plot_distribution(total_df, bins=100, title="Distribution")

In [ ]:
arparams = np.array([.75, -.25])
maparams = np.array([.65, .35])
ar = np.r_[1, -arparams] # add zero-lag and negate
ma = np.r_[1, maparams] # add zero-lag
arma_process = sm.tsa.ArmaProcess(ar, ma)

In [ ]:
plot_inverse_arma_roots(arma_process)

In [ ]:
plot_extended_scatter_matrix(total_df.dropna(), y_cols=["y", "x_1", "x_2"])

In [ ]:
plot_ts_overview(ts_df)

In [ ]:
plot_ts_analysis(ts_df)

In [ ]:
plot_gof(total_df, "y", "y_hat")